In [1]:
from konlpy.tag import Mecab
from pykospacing import Spacing
mecab = Mecab()
print(mecab.morphs(u'영등포구청역에 있는 맛집 좀 알려주세요.'))

['영등포구청역', '에', '있', '는', '맛집', '좀', '알려', '주', '세요', '.']


# 텍스트 전처리 (Text Preprocessing)

*   텍스트를 자연어 처리를 위해 용도에 맞도록 사전에 표준화 하는 작업
*   텍스트 내 정보를 유지하고, 중복을 제거하여 분석 효율성을 높이기 위해 전처리를 수행



### 1) 토큰화 (Tokenizing)
* 텍스트를 자연어 처리를 위해 분리 하는 것을
* 토큰화는 단어별로 분리하는 "단어 토큰화(Word Tokenization)"와 문장별로 분리하는 "문장 토큰화(Sentence Tokenization)"로 구분

(이후 실습에서는 단어 토큰화를 "토큰화"로 통일하여 칭하도록 한다)

### 2) 품사 부착(PoS Tagging)
* 각 토큰에 품사 정보를 추가
* 분석시에 불필요한 품사를 제거하거나 (예. 조사, 접속사 등) 필요한 품사를 필터링 하기 위해 사용

### 3) 개체명 인식 (NER, Named Entity Recognition)
* 각 토큰의 개체 구분(기관, 인물, 지역, 날짜 등) 태그를 부착
* 텍스트가 무엇과 관련되어있는지 구분하기 위해 사용
* 예를 들어, 과일의 apple과 기업의 apple을 구분하는 방법이 개체명 인식임

### 4) 원형 복원 (Stemming & Lemmatization)
* 각 토큰의 원형 복원을 함으로써 토큰을 표준화하여 불필요한 데이터 중복을 방지 (=단어의 수를 줄일수 있어 연산을 효율성을 높임)
* 어간 추출(Stemming) : 품사를 무시하고 규칙에 기반하여 어간을 추출
* 표제어 추출 (Lemmatization) : 품사정보를 유지하여 표제어 추출

### 5) 불용어 처리 (Stopword)
* 자연어 처리를 위해 불필요한 요소를 제거하는 작업
* 불필요한 품사를 제거하는 작업과 불필요한 단어를 제거하는 작업으로 구성
* 불필요한 토큰을 제거함으로써 연산의 효율성을 높임



---



# 1 영문 전처리 실습


NLTK lib (https://www.nltk.org/) 사용

## 1.1 실습용 영문기사 수집
온라인 기사를 바로 수집하여 실습데이터로 사용

https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/

In [1]:
import requests
from bs4 import BeautifulSoup
url = "https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/"
res = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0'})
soup =BeautifulSoup(res.text, 'html.parser')

In [3]:
article = soup.select('p')
text = article[3].get_text()
text

"And yes, she does mean everybody's job from yours to mine and onward to the role of grain farmers in Egypt, pastry chefs in Paris and dog walkers in Oregon i.e. every job. We will now be able to help direct all workers’ actions and behavior with a new degree of intelligence that comes from predictive analytics, all stemming from the AI engines we will now increasingly depend upon."

## 1.2 영문 토큰화
https://www.nltk.org/api/nltk.tokenize.html

In [4]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /Users/dhkim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
word_tokens = word_tokenize(text)
print(word_tokens)

['And', 'yes', ',', 'she', 'does', 'mean', 'everybody', "'s", 'job', 'from', 'yours', 'to', 'mine', 'and', 'onward', 'to', 'the', 'role', 'of', 'grain', 'farmers', 'in', 'Egypt', ',', 'pastry', 'chefs', 'in', 'Paris', 'and', 'dog', 'walkers', 'in', 'Oregon', 'i.e', '.', 'every', 'job', '.', 'We', 'will', 'now', 'be', 'able', 'to', 'help', 'direct', 'all', 'workers', '’', 'actions', 'and', 'behavior', 'with', 'a', 'new', 'degree', 'of', 'intelligence', 'that', 'comes', 'from', 'predictive', 'analytics', ',', 'all', 'stemming', 'from', 'the', 'AI', 'engines', 'we', 'will', 'now', 'increasingly', 'depend', 'upon', '.']


In [6]:
word_tokens = word_tokenize("Good muffins cost $3.88\nin New York")
print(word_tokens)

['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York']


In [7]:
from nltk.tokenize import WordPunctTokenizer
word_tokens = WordPunctTokenizer().tokenize("Good muffins cost $3.88\nin New York")
print(word_tokens)

['Good', 'muffins', 'cost', '$', '3', '.', '88', 'in', 'New', 'York']


In [8]:
from nltk.tokenize import TreebankWordTokenizer
word_tokens = TreebankWordTokenizer().tokenize("Good muffins cost $3.88\nin New York")
print(word_tokens)


## 또는 spicy 를 사용하여 tokenize 를 할 수 있다.

['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York']


## 1.3 영문 품사 부착 (PoS Tagging)
분리한 토큰마다 품사를 부착한다

https://www.nltk.org/api/nltk.tag.html

태크목록 : https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

In [9]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dhkim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [10]:
tagged = pos_tag(word_tokens)
print(tagged)

[('Good', 'JJ'), ('muffins', 'NNS'), ('cost', 'VBP'), ('$', '$'), ('3.88', 'CD'), ('in', 'IN'), ('New', 'NNP'), ('York', 'NNP')]


## 1.4 개체명 인식 (NER, Named Entity Recognition)

http://www.nltk.org/api/nltk.chunk.html

In [11]:
nltk.download("words")
nltk.download("maxent_ne_chunker")
# 한국어는 선택사항인듯..





[nltk_data] Downloading package words to /Users/dhkim/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/dhkim/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [12]:
from nltk import ne_chunk
ne_token = ne_chunk(tagged)
print(ne_token)
# spicy 는 앞의 과정을 한번에 해준다.

(S
  (GPE Good/JJ)
  muffins/NNS
  cost/VBP
  $/$
  3.88/CD
  in/IN
  (GPE New/NNP York/NNP))


## 1.5 원형 복원
각 토큰의 원형을 복원하여 표준화 한다. 

### 1.5.1 어간추출 (Stemming)

* 규칙에 기반 하여 토큰을 표준화
* ning제거, ful 제거 등

https://www.nltk.org/api/nltk.stem.html

규칙상세 : https://tartarus.org/martin/PorterStemmer/def.txt

In [13]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()



In [14]:
ps.stem("running")

'run'

In [15]:
ps.stem("beautiful")

'beauti'

In [16]:
ps.stem("believes")

'believ'

### 1.5.2 표제어 추출 (Lemmatization)

* 품사정보를 보존하여 토큰을 표준화

http://www.nltk.org/api/nltk.stem.html?highlight=lemmatizer

In [17]:
#표준화의 한 방법
nltk.download("wordnet")



[nltk_data] Downloading package wordnet to /Users/dhkim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()



In [19]:
wl.lemmatize("running")

'running'

In [20]:
wl.lemmatize("beautiful")

'beautiful'

In [21]:
wl.lemmatize("believes")

'belief'

## 1.6 불용어 처리 (Stopword)

In [22]:
# 분석을 할 때 제거하는것 
# 사전을 활용하는방법
stop_pos = ['IN','CC','DT']
from collections import Counter


In [23]:
tagged = pos_tag(word_tokenize(text))
Counter(tagged).most_common()

[((',', ','), 3),
 (('from', 'IN'), 3),
 (('to', 'TO'), 3),
 (('and', 'CC'), 3),
 (('in', 'IN'), 3),
 (('.', '.'), 3),
 (('job', 'NN'), 2),
 (('the', 'DT'), 2),
 (('of', 'IN'), 2),
 (('will', 'MD'), 2),
 (('now', 'RB'), 2),
 (('all', 'DT'), 2),
 (('And', 'CC'), 1),
 (('yes', 'UH'), 1),
 (('she', 'PRP'), 1),
 (('does', 'VBZ'), 1),
 (('mean', 'VB'), 1),
 (('everybody', 'NN'), 1),
 (("'s", 'POS'), 1),
 (('yours', 'NNS'), 1),
 (('mine', 'VB'), 1),
 (('onward', 'VB'), 1),
 (('role', 'NN'), 1),
 (('grain', 'NN'), 1),
 (('farmers', 'NNS'), 1),
 (('Egypt', 'NNP'), 1),
 (('pastry', 'NN'), 1),
 (('chefs', 'NNS'), 1),
 (('Paris', 'NNP'), 1),
 (('dog', 'NN'), 1),
 (('walkers', 'NNS'), 1),
 (('Oregon', 'NNP'), 1),
 (('i.e', 'NN'), 1),
 (('every', 'DT'), 1),
 (('We', 'PRP'), 1),
 (('be', 'VB'), 1),
 (('able', 'JJ'), 1),
 (('help', 'VB'), 1),
 (('direct', 'VB'), 1),
 (('workers', 'NNS'), 1),
 (('’', 'VBP'), 1),
 (('actions', 'NNS'), 1),
 (('behavior', 'NN'), 1),
 (('with', 'IN'), 1),
 (('a', 'DT'), 1

In [24]:
words = []
for tag in tagged:
    if not tag[1] in stop_pos:
        words.append(tag[0])
        
print(words)

['yes', ',', 'she', 'does', 'mean', 'everybody', "'s", 'job', 'yours', 'to', 'mine', 'onward', 'to', 'role', 'grain', 'farmers', 'Egypt', ',', 'pastry', 'chefs', 'Paris', 'dog', 'walkers', 'Oregon', 'i.e', '.', 'job', '.', 'We', 'will', 'now', 'be', 'able', 'to', 'help', 'direct', 'workers', '’', 'actions', 'behavior', 'new', 'degree', 'intelligence', 'that', 'comes', 'predictive', 'analytics', ',', 'stemming', 'AI', 'engines', 'we', 'will', 'now', 'increasingly', 'depend', 'upon', '.']


## 1.7 영문 텍스트 전처리 종합



---



# 2 한글 전처리 실습
영문은 공백으로 토큰화가 가능하지만, 한글의 경우 품사를 고려하여 토큰화 해야한다.

## 2.1 실습용 한글기사 수집
온라인 기사를 바로 수집하여 실습데이터로 사용

http://news.chosun.com/site/data/html_dir/2018/07/10/2018071004121.html

In [25]:
url = "https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=110&oid=023&aid=0003386456"
headers = {'user-agent':'Mozilla/5.0'}
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')


In [26]:
article = soup.select("#articleBodyContents")[0].text

type(article)

str

## 2.2 한글 토큰화 및 형태소 분석

한글 자연어처리기 비교

https://konlpy.org/ko/latest/morph/

In [29]:
text = "나나나 나 나나나나나 나나나 나 나나나나나 나나나 나 나나나나나"
from konlpy.tag import Komoran
komoran = Komoran()

tagged = komoran.morphs(text)
tagged

['나나', '나', '나', '나나', '나나', '나']

## 2.3 한글 품사 부착 (PoS Tagging)

PoS Tag 목록

https://docs.google.com/spreadsheets/u/1/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0

In [36]:
pos = mecab.pos(article)
tagged=pos

## 2.4 불용어(Stopword) 처리
분석에 불필요한 품사를 제거하고, 불필요한 단어(불용어)를 제거한다

In [37]:
stopwords = []
with open("stopwords.txt") as f:
    for line in f:
        stopwords.append(line.strip())
len(stopwords)

675

In [40]:
stop_pos = ['EC','EP','SC','JK','JKO']

In [41]:
print(len(tagged))
words=[]
for tag in tagged:
    if tag[0] in stopwords or tag[1] in stop_pos:
        continue
    words.append(tag[0])
    
print(len(words))

1119
998


# 2 N-gram